# Maciej Legas 2031545 Coursework 1

In [31]:
import csv
import itertools as itr
from timeit import default_timer as timer

## Basic I/O functions
Below are three functions for reading the dataset, storing evaluation results and finding unique values in a list.

In [32]:
def read_dataset(data_file):
    filtered_dataset = []
    
    with open(data_file) as data:
        read_data = csv.reader(data)
        for row in read_data:
            # The row below removes empty values
            filtered_data = list(filter(None, row))
            filtered_dataset.append(filtered_data)
        
    return filtered_dataset

In [33]:
def store_results(results):
    results_file = open("results.txt", "w")
    results_file.write("\n".join(str(row) for row in results))
    results_file.close()

In [34]:
def unique_values(dataset):
    unique_list = []

    for row in dataset:
        for value in row:
            if value not in unique_list:
                unique_list.append(value)

    return unique_list

In [35]:
data = read_dataset('GroceryStore.csv')

## Frequent Itemset

Brute-force approach, no pruning. Computationally expensive as we are considering all possible itemset combinations as candidates.

In [36]:
def frequent_itemset(dataset, min_sup, itemset_limit=5):
    itemsets = []
    # Dictionary: Itemset as key - Support as value
    supports = {}
    # Dictionary: Itemset as key - Frequency as value
    frequencies = {}
    transaction_count = len(dataset)

    # Find unique values (1-itemsets)
    unique_list = unique_values(dataset)

    if itemset_limit < 0 or itemset_limit > len(unique_list):
        print("Illegal value set as the itemset limit, returning to the default behaviour.")
        itemset_limit = 0

    if itemset_limit == 0:
        itemset_limit = len(unique_list)

    else:
        itemset_limit = itemset_limit + 1

    for i in range(1, itemset_limit):
        # Use itertools to find all possible combinations of the unique values without repetitions (order does not matter as well), with i being the value of r
        combinations = itr.combinations(unique_list, i)
        itemset = []

        for combination in combinations:        
            appearance_count = 0

            for row in dataset:
                # Check if a row of the given dataset contains all values of the combination
                if all(value in row for value in combination):
                    appearance_count = appearance_count + 1

            support = appearance_count / transaction_count

            # Store where an itemset appears in the dataset
            frequencies[combination] = appearance_count

            if support >= min_sup:
                # Store rows in which the combination appears
                itemset.append(combination)
                # Store the supports of an itemset
                supports[combination] = support

        itemset.sort()
        itemsets.append(itemset)

    return frequencies, itemsets, supports

In [37]:
frequencies, itemsets, supports = frequent_itemset(dataset=data, min_sup=0.1, itemset_limit=5)

In [38]:
def print_itemsets(itemsets, supports):
    count = 1
    print("--------------------------------------")
    for itemset_row in itemsets:
        if not itemset_row:
            print("No {}-itemsets have been found.".format(count))
            print("Try to lower the minimum support value if that was not intended.")
        else:
            print("Possible {}-itemsets with supports:".format(count))
            for itemset in itemset_row:
                print(itemset, supports[itemset])
        print("--------------------------------------")
        count = count + 1

Below are the found 2, 3, 4 and 5-frequent itemsets.

In [39]:
print_itemsets(itemsets, supports)

--------------------------------------
Possible 1-itemsets with supports:
('Bread',) 0.43780935653840014
('Butter',) 0.4375698547022194
('Cheese',) 0.43717068497525147
('Coffee Powder',) 0.4398052051732397
('Ghee',) 0.43988503911863325
('Lassi',) 0.43365799137793387
('Milk',) 0.44116238224493054
('Panner',) 0.4346159987226569
('Sugar',) 0.43764968864761294
('Sweet',) 0.43772952259300657
('Tea Powder',) 0.4297461280536484
('Yougurt',) 0.4393262015008782
--------------------------------------
Possible 2-itemsets with supports:
('Bread', 'Milk') 0.20094204055564427
('Bread', 'Sugar') 0.19790835063068818
('Bread', 'Sweet') 0.20269838735430304
('Butter', 'Bread') 0.1969503432859652
('Butter', 'Cheese') 0.19838735430304966
('Butter', 'Ghee') 0.20197988184576082
('Butter', 'Milk') 0.19886635797541113
('Butter', 'Panner') 0.1982276864122625
('Butter', 'Sugar') 0.20525307360689765
('Butter', 'Sweet') 0.20301772313587738
('Butter', 'Tea Powder') 0.19295864601628612
('Butter', 'Yougurt') 0.201900

## Finding associated rules
This method has been optimized to use stored frequencies in calculating confidence, running way faster than searching each time for set intersections for each X and Y in the dataset.

The below two functions have been implemented in order to help with generating candidate rules of appropriate length. By this, we mean that there is always a set of X and Y and not above.

In [40]:
# Helper recursive function adapted and modified from source: https://stackoverflow.com/questions/19368375/set-partitions-in-python
def partition(itemset):
    if len(itemset) == 1:
        yield [ itemset ]
        return

    first = itemset[0]
    for smaller in partition(itemset[1:]):
        for n, subset in enumerate(smaller):
            yield smaller[:n] + [[ first ] + subset]  + smaller[n+1:]
        yield [[ first ]] + smaller

In [41]:
def generate_candidate_rules(itemset):
    candidate_rules = []

    if len(itemset) <= 1:
        return itemset

    else:
        partitioned_itemset = partition(itemset)
    
    for items in partitioned_itemset:
        if len(items) != 2:
            # For candidate rules we require to have all items in two subsets
            # We are rejecting subsets with multiple single items such as ['A', 'B', 'CD']
            continue
        else:
            candidate_rules.append(items)
            # Reverse the order for reverse rules, for example: A -> BC, BC -> A
            candidate_rules.append(items[::-1])
    
    return candidate_rules

Test showcase of how the functions above work.

In [42]:
test_list = ["A", "B", "C", "D"]

for value in generate_candidate_rules(test_list):
    print(value)

[['A'], ['B', 'C', 'D']]
[['B', 'C', 'D'], ['A']]
[['A', 'B'], ['C', 'D']]
[['C', 'D'], ['A', 'B']]
[['B'], ['A', 'C', 'D']]
[['A', 'C', 'D'], ['B']]
[['A', 'B', 'C'], ['D']]
[['D'], ['A', 'B', 'C']]
[['B', 'C'], ['A', 'D']]
[['A', 'D'], ['B', 'C']]
[['A', 'C'], ['B', 'D']]
[['B', 'D'], ['A', 'C']]
[['C'], ['A', 'B', 'D']]
[['A', 'B', 'D'], ['C']]


In [43]:
def mine_association_rules(dataset, frequencies, itemsets, supports, min_conf):
    # List for generated rules
    output = [[], [], [], []]
    confidences = []
    transaction_count = len(dataset)
    # Removing empty itemsets
    filtered_itemsets = list(filter(None, itemsets))

    # Iterate only through itemsets with a collection of 2 or above, no use of going through 1-itemsets
    i = len(filtered_itemsets) - 1

    while i > 0:
        for itemset in filtered_itemsets[i]:
            candidate_rules = generate_candidate_rules(list(itemset))
            
            for row in candidate_rules:
                # Find where in the dataset both items appear
                combined_x_and_y = row[0] + row[1]
                key = combined_x_and_y

                if tuple(key) not in frequencies:
                    # Try to sort the key and see if we can find it before iterating through all the keys
                    key.sort()
                    if tuple(key) not in frequencies:
                        # We try to find the key containing the values while being of the same length
                        for dictionary_key in frequencies.keys():
                            if all(value in dictionary_key for value in key) and len(dictionary_key) == len(key):
                                key = dictionary_key

                x_and_y_frequencies = frequencies[tuple(key)]
                x_freq = frequencies[tuple(row[0])]

                # Calculate confidence: X and Y frequencies / X frequency
                confidence = x_and_y_frequencies / x_freq
                if confidence >= min_conf:
                    output[0].append(row[0])
                    output[1].append(row[1])
                    output[2].append(supports[itemset])
                    output[3].append(confidence)

        i = i - 1

    return output

In [44]:
output = mine_association_rules(data, frequencies, itemsets, supports, min_conf=0.5)

In [45]:
def print_association_rules(output):
    if not output[0]:
        print("No association rules have been found.")
    else:
        print("Association rules found:")
        for row in range(len(output[0])):
            print("{} => {} | Support: {} | Confidence: {}".format(output[0][row], output[1][row], output[2][row], output[3][row]))

In [46]:
print_association_rules(output)

Association rules found:
['Panner', 'Sweet'] => ['Lassi'] | Support: 0.10098994092288041 | Confidence: 0.5049900199600799
['Lassi', 'Panner'] => ['Sweet'] | Support: 0.10098994092288041 | Confidence: 0.5066079295154186


## Apriori algorithm
Below is the implementation of the apriori algorithm. The main repeating functions have been separated from the main function to separate ones.

In [47]:
def generate_apriori_candidates(itemsets, k):
    candidate_set = []
    # If we do not need to meet any prefix criteria, just generate a combination
    if k - 2 < 0:
        combinations = itr.combinations(itemsets, k+1)
        for combination in combinations:
            candidate_set.append(combination)
    else:
        count = 0
        for item in itemsets:
            for row in range(count, len(itemsets)):
                if item == itemsets[row]:
                    continue
                else:
                    prefix_count = 0
                    merged_itemset = []
                    
                    for prefix in range(k-1):
                        if item[prefix] == itemsets[row][prefix]:
                            prefix_count = prefix_count + 1

                    if prefix_count == k-1:
                        for prefix in range(k-1):
                            merged_itemset.append(item[prefix])
                        merged_itemset.append(item[k-1])
                        merged_itemset.append(itemsets[row][k-1])
                        candidate_set.append(merged_itemset)
                        
            count = count + 1
            
    return candidate_set

In [48]:
def prune_apriori_candidates(candidate_set, supports, min_sup, k):
    pruned_candidate_set = []

    for candidate in candidate_set:
        filtered_candidate_set = []
        # Partition the generated candidate set
        partitions = partition(list(candidate))
        for subsets in partitions:
            for subset in subsets:
                # Leave only subsets of length K
                if len(subset) == k:
                    if subset not in filtered_candidate_set:    
                        filtered_candidate_set.append(subset)

        for sets in filtered_candidate_set:
            for subset in sets:
                if type(subset) is not tuple:
                    subset = (subset,)
                if supports[subset] >= min_sup:
                    # Eliminate subsets of length K that are not above the minimum support criteria
                    if sets not in pruned_candidate_set:
                        pruned_candidate_set.append(sets)

    return pruned_candidate_set

In [49]:
def candidate_elimination_apriori(candidate_set, dataset, frequencies, supports, min_sup, k):
    transaction_count = len(dataset)
    itemset = []
    
    for candidate in candidate_set:
        item_positions = []            
        appearance_count = 0

        # This row converts the tuple of tuples of strings into a single tuple of strings
        # It is required to get the result from itertools.combinations properly iterated
        if k == 1:
            candidate = tuple(itr.chain(*candidate))

        # Else just convert the list of candidates to a tuple
        else:
            candidate = tuple(candidate)

        for row_count, row in enumerate(dataset):
            # Check if a row of the given dataset contains all values of the candidate
            if all(value in row for value in candidate):
                appearance_count = appearance_count + 1
                # Collect in which rows the candidate appears
                item_positions.append(row_count)

        support = appearance_count / transaction_count

        # Store where a candidate appears in the dataset
        frequencies[candidate] = appearance_count

        if support > min_sup:
            # Store the frequent candidate
            itemset.append(candidate)
            # Store the support of the itemset
            supports[candidate] = support

    itemset.sort()

    return frequencies, itemset, supports

Below is the main algorithm function.

In [50]:
def apriori(dataset, min_sup):
    itemsets = []
    # Dictionary: Itemset as key - Support as value
    supports = {}
    # Dictionary: Itemset as key - Frequency as value
    frequencies = {}
    transaction_count = len(dataset)

    # Find unique values (1-itemsets)
    unique_list = unique_values(dataset)

    # First step of the algorithm, generation of frequent 1-itemsets
    
    # Although we do not need to find any combinations for frequent 1-itemsets, the use of the combinations function ensures the itemsets always stay as the type of tuples, required for storing them in dictionaries.
    combinations = itr.combinations(unique_list, 1)
    itemset = []

    for combination in combinations:          
        appearance_count = 0

        for row in dataset:
            # Check if a row of the given dataset contains all values of the combination
            if all(value in row for value in combination):
                appearance_count = appearance_count + 1

        support = appearance_count / transaction_count

        # Store the support of the itemset
        # We need to store all supports of the 1-itemsets for candidate pruning
        supports[combination] = support

        # Store where an itemset appears in the dataset
        frequencies[combination] = appearance_count

        if support >= min_sup:
            # Store the 1-frequent itemset
            itemset.append(combination)

    itemset.sort()
    # Store the frequent 1-itemsets
    itemsets.append(itemset)

    # Second step of the algorithm: repeating until we run out of frequent itemsets

    # Algorithm "k" variable
    # While it should be set to 2 here, we start k at 1 to
    # ensure that iterating through a list remains simple.
    k = 1

    while itemsets[k-1] != []:
        # Generate a candidate set
        candidate_set = generate_apriori_candidates(itemsets[k-1], k)
        # Prune candidate itemsets
        pruned_candidate_set = prune_apriori_candidates(candidate_set, supports, min_sup, k)
        # Count the support and eliminate infrequent itemsets, returning a new itemset
        frequencies, new_itemset, supports = candidate_elimination_apriori(candidate_set, dataset, frequencies, supports, min_sup, k)

        itemsets.append(new_itemset)
        k = k + 1

    return frequencies, itemsets, supports

In [51]:
frequencies, itemsets, supports = apriori(dataset=data, min_sup=0.1)

Below are the found itemsets using the apriori algorithm.

In [52]:
print_itemsets(itemsets, supports)

--------------------------------------
Possible 1-itemsets with supports:
('Bread',) 0.43780935653840014
('Butter',) 0.4375698547022194
('Cheese',) 0.43717068497525147
('Coffee Powder',) 0.4398052051732397
('Ghee',) 0.43988503911863325
('Lassi',) 0.43365799137793387
('Milk',) 0.44116238224493054
('Panner',) 0.4346159987226569
('Sugar',) 0.43764968864761294
('Sweet',) 0.43772952259300657
('Tea Powder',) 0.4297461280536484
('Yougurt',) 0.4393262015008782
--------------------------------------
Possible 2-itemsets with supports:
('Bread', 'Butter') 0.1969503432859652
('Bread', 'Cheese') 0.20197988184576082
('Bread', 'Coffee Powder') 0.20182021395497365
('Bread', 'Ghee') 0.19982436532013412
('Bread', 'Lassi') 0.20006386715631486
('Bread', 'Milk') 0.20094204055564427
('Bread', 'Panner') 0.20357656075363245
('Bread', 'Sugar') 0.19790835063068818
('Bread', 'Sweet') 0.20269838735430304
('Bread', 'Tea Powder') 0.1951939964873064
('Bread', 'Yougurt') 0.20014370110170845
('Butter', 'Cheese') 0.198

In [53]:
output = mine_association_rules(data, frequencies, itemsets, supports, 0.5)

In [54]:
print_association_rules(output)

Association rules found:
['Panner', 'Sweet'] => ['Lassi'] | Support: 0.10098994092288041 | Confidence: 0.5049900199600799
['Lassi', 'Panner'] => ['Sweet'] | Support: 0.10098994092288041 | Confidence: 0.5066079295154186


## FP-Growth algorithm
This implementation is NOT finished, therefore it will NOT be taken upon further evaluation, although it does have the grounds for further development, such as the creation of the Node class and successful pass of the first step of the algorithm (FP-Tree construction).

In [55]:
class Node:
    item_name = None
    parent = None
    children = []
    linked_node = None
    count = 1

    def __init__(self, item_name, parent):
        self.item_name = item_name
        self.parent = parent

In [56]:
def fp_growth(dataset, min_sup):
    itemsets = []
    # Dictionary: Itemset as key - Support as value
    supports = {}
    # Dictionary: Itemset as key - Frequency as value
    frequencies = {}
    transaction_count = len(dataset)

    # Find unique values (1-itemsets)
    unique_list = unique_values(dataset)

    # First step of the algorithm, generation of frequent 1-itemsets
    # First pass over the dataset
    
    # Although we do not need to find any combinations for frequent 1-itemsets, the use of the combinations function ensures the itemsets always stay as the type of tuples, required for storing them in dictionaries.
    combinations = itr.combinations(unique_list, 1)
    itemset = []

    for combination in combinations:          
        appearance_count = 0

        for row in dataset:
            # Check if a row of the given dataset contains all values of the combination
            if all(value in row for value in combination):
                appearance_count = appearance_count + 1

        support = appearance_count / transaction_count

        # Store the support of the itemset
        supports[combination] = support

        if support >= min_sup:
            # Store where an itemset appears in the dataset
            frequencies[combination] = appearance_count

    # Sort (descending) the frequent 1-itemsets according to frequency
    sorted_frequencies = sorted(frequencies.items(), key=lambda i: i[1], reverse=True)

    itemset = []
    for i in sorted_frequencies:
        itemset.append(i[0])

    # Pass 2 of the dataset
    # We create the FP-Tree here
    fp_tree = [[] for _ in range(len(itemset))]

    for row in dataset:
        previous_nodes = []
        for item in itemset:
            # Check if a row of the given dataset contains the 1-itemset
            if all(value in tuple(row) for value in item):
                # Check if there are no parents, meaning we start at root
                if not previous_nodes:
                    # Create a node with "null" root being the parent
                    node = Node(item, "null")
                    previous_nodes.append(node)
                else:
                    # We already have previously found nodes with higher frequencies
                    # We need to update the child of the last node
                    node = Node(item, previous_nodes[-1])
                    previous_nodes[-1].child = node
                    previous_nodes.append(node)
            
        # Compare here the previous nodes with the FP-tree and add counts or create new nodes
        if previous_nodes:
            count = 0
            for depth_level in previous_nodes:
                node_not_existing_yet = True
                if fp_tree[count]:
                    for node in fp_tree[count]:
                        if depth_level.item_name == node.item_name:
                            node.count = node.count + 1
                            
                            if depth_level.children != node.children:
                                node.children.append(depth_level.children)

                            node_not_existing_yet = False
                            break
                    if node_not_existing_yet == True:
                        fp_tree[count].append(depth_level)

                else:
                    fp_tree[count].append(depth_level)
                count = count + 1

    
    # We need to find head pointers for linking nodes.
    header_table = {}

    for item in itemset:
        header_pointer_found = False
        while header_pointer_found == False:
            for column in range(len(fp_tree)-1, -1, -1):
                for node in fp_tree[column]:
                    if node.item_name == item:
                        # If we do not have a head pointer yet for this item, set the deepest found node as it
                        if header_pointer_found == False:
                            header_table[item] = node
                            header_pointer_found = True
                        # If we do have a set head pointer already for this item, use this node as a linked node
                        else:
                            main_node = header_table[item]
                            linked_node_set = False
                            temp_node = main_node.linked_node

                            while linked_node_set == False:
                                if temp_node is not None:
                                    temp_node = temp_node.linked_node
                                else:
                                    temp_node = node
                                    linked_node_set = True
                                    
                            
    return fp_tree
        # Pass 2 finished, we have a FP-Tree
        # Lacking - Step 2: Frequent Itemset Generation.

In [57]:
fp_tree = fp_growth(dataset=data, min_sup=0.1)

## Experiment on the dataset, compare run-time performance
Below we will run both brute-force and apriori algorithms on minimum support values of 0.4, 0.2, 0.1 and 0.05, with a minimum confidence value of 0.5. The brute-force algorithm will be limited at 8-itemsets, as it is expected that with a higher itemset limit, the runtime will grow massively.

In [65]:
def compare_times(minimum_support, minimum_confidence, itemset_limit_value):
    start = timer()
    frequencies, itemsets, supports = frequent_itemset(dataset=data, min_sup=minimum_support, itemset_limit=itemset_limit_value)
    bruteforce_output = mine_association_rules(data, frequencies, itemsets, supports, minimum_confidence)
    end = timer()
    bruteforce_time = end - start

    start = timer()
    frequencies, itemsets, supports = apriori(dataset=data, min_sup=minimum_support)
    apriori_output = mine_association_rules(data, frequencies, itemsets, supports, minimum_confidence)
    end = timer()
    apriori_time = end - start

    results = []
    results.append("------------------------------------------------------------")
    results.append("Evaluating times for apriori and bruteforce algorithms:")
    results.append("Minimum support: {} ||| Minimum confidence: {} ||| Itemset limit for bruteforce: {}".format(minimum_support, minimum_confidence, itemset_limit_value))
    results.append("")
    results.append("Time taken for the bruteforce algorithm: {} seconds".format(bruteforce_time))
    if not bruteforce_output[0]:
        results.append("No association rules have been found.")
    else:
        results.append("Association rules found:")
        for row in range(len(bruteforce_output[0])):
            results.append("{} => {} | Support: {} | Confidence: {}".format(bruteforce_output[0][row], bruteforce_output[1][row], bruteforce_output[2][row], bruteforce_output[3][row]))
    results.append("")
    results.append("Time taken for the apriori algorithm: {} seconds".format(apriori_time))
    if not apriori_output[0]:
        results.append("No association rules have been found.")
    else:
        results.append("Association rules found:")
        for row in range(len(apriori_output[0])):
            results.append("{} => {} | Support: {} | Confidence: {}".format(apriori_output[0][row], apriori_output[1][row], apriori_output[2][row], apriori_output[3][row]))
    results.append("")

    return results

In [67]:
results = [[], [], [], [], []]

results[0] = compare_times(0.4, 0.5, 8)
results[1] = compare_times(0.2, 0.5, 8)
results[2] = compare_times(0.1, 0.5, 8)
results[3] = compare_times(0.05, 0.5, 8)

file_ready_results = []

for column in results:
    for row in column:
        print(row)
        file_ready_results.append(row)

------------------------------------------------------------
Evaluating times for apriori and bruteforce algorithms:
Minimum support: 0.4 ||| Minimum confidence: 0.5 ||| Itemset limit for bruteforce: 8

Time taken for the bruteforce algorithm: 71.40586452599973 seconds
No association rules have been found.

Time taken for the apriori algorithm: 1.4229402240002855 seconds
No association rules have been found.

------------------------------------------------------------
Evaluating times for apriori and bruteforce algorithms:
Minimum support: 0.2 ||| Minimum confidence: 0.5 ||| Itemset limit for bruteforce: 8

Time taken for the bruteforce algorithm: 76.0077230689999 seconds
No association rules have been found.

Time taken for the apriori algorithm: 2.4180687809998744 seconds
No association rules have been found.

------------------------------------------------------------
Evaluating times for apriori and bruteforce algorithms:
Minimum support: 0.1 ||| Minimum confidence: 0.5 ||| Items

### Interesting association rules
As we can see above, there is a small amount of association rules found.

Customers containing Sugar and Sweet tend to buy Butter, while transactions containing Panner and Sweet tend to have Lassi as well. The latter is vice versa as well - transactions containing Lassi and Panner show a tendency to have Sweet in the transaction. 

In terms of evaluation, both algorithms are accurate in finding association rules.

### Performance evaluation
The a priori algorithm beats the brute-force in terms of time efficiency at each trial (0.4, 0.2, 0.1, 0.05 minimum support). 

However, while the time taken for the brute-force algorithm remains relatively constant, as it solely depends on the itemset limit due to treating all itemsets as possible candidate rules, there is a clear scalability issue shown by the a priori algorithm (1.42s -> 2.41s -> 9.32s -> 19.51s), which is due to the fact that the a priori algorithm has to traverse through the dataset each time in its main loop. Since the number of the potential candidate rules rises when the minimum support value lowers, we can see how time complex this algorithm is. A potential solution would be to use algorithms that traverse through the dataset as little as possible, such as the FP-Growth algorithm.

In [68]:
store_results(file_ready_results)